In [1]:
# in general: sample random negative examples from the CR data, assuming they are not verse references; then pick top verse match according to all three methods

# imports
import pandas as pd
import spacy
import time
from tqdm import tqdm
import sys
import argparse
import os
import json
import re
import nltk

from data import congress_utils, data_utils
from src.references.retrieve_and_infer import load_bible_data_for_references, create_inference_df


BIBLE_VERSION = 'King James Bible'
OUT_DATA_PATH = '/home/laviniad/projects/religion_in_congress/src/references/validation/data/true_positive_examples.csv'
MIN_SENTENCE_LENGTH = 5

VERSE_REGEX = r'\b(?:\d\s\w+\s\d+:\d+-\d+|\d\s\w+\s\d+:\d+|\w+\s\d+:\d+-\d+|\w+\s\d+:\d+)\b'


[nltk_data] Downloading package punkt to /home/laviniad/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
verse_df, limited_verse_to_citation, limited_citation_to_verse = load_bible_data_for_references(version=BIBLE_VERSION)
cr_df = congress_utils.load_full_df_from_raw('/data/corpora/congressional-record/')

# transform into sentence-wise df
cr_df = create_inference_df(MIN_SENTENCE_LENGTH, cr_df)

Loading Bible data for version: King James Bible
Loading verses...
Number of verses: 1000
On file 0


100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 106493/106493 [00:00<00:00, 202036.55it/s]


On file 1


100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 190933/190933 [00:00<00:00, 201666.92it/s]


On file 2


...

100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 210379/210379 [00:01<00:00, 199635.41it/s]


On file 17


100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 196111/196111 [00:00<00:00, 201530.50it/s]


On file 18


100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 2405671/2405671 [11:21<00:00, 3528.20it/s]


In [8]:
from rapidfuzz import fuzz

def find_bible_citations(cr_df, limited_citation_to_verse):
    ref_list = []

    for idx, row in tqdm(cr_df.iterrows()):
        congress_text = row['text']
    
        verse_citations = re.findall(VERSE_REGEX, congress_text)   

        for citation in verse_citations:
            citation = citation.lower()
            if citation in limited_citation_to_verse:
                ref_list.append((row['congress_idx'], congress_text, citation, 1))
                print("Found Bible verse citation: ", citation)

    return ref_list

In [ ]:
true_positive_examples = find_bible_citations(cr_df, limited_citation_to_verse)
print("True positive examples found via regex: ", len(true_positive_examples))    
true_positive_examples.to_csv(OUT_DATA_PATH, index=False)
print("Saved true positive examples to ", OUT_DATA_PATH)


In [15]:
true_positive_examples = pd.DataFrame(true_positive_examples, columns=['congress_idx', 'text', 'verse', 'label'])

In [16]:
true_positive_examples.to_csv(OUT_DATA_PATH, index=False)